In [2]:
cd ../

/app


/home/arun/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv("./data/raw/jigsaw-toxic-comments/train.csv")
test_df = pd.read_csv("./data/raw/jigsaw-toxic-comments/test.csv")
test_labels = pd.read_csv("./data/raw/jigsaw-toxic-comments/test_labels.csv")

In [5]:
print(f"{train_df.shape=}")
print(f"{test_df.shape=}")
print(f"{test_labels.shape=}")

train_df.shape=(159571, 8)
test_df.shape=(153164, 2)
test_labels.shape=(153164, 7)


> This dataset is a large one. Also there are lot of data in test set, we might not use all the data points, instead 
put some data from test set into train dataset for processing

In [6]:
train_df.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


**So in here same as ghc dataset, we have 6 labels which indicates offensive text and its level. So it is good to combine these labels into single label by rule such as if one of the column is one, the final label is one**

In [6]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [8]:
train_df['toxic'].value_counts()

toxic
0    144277
1     15294
Name: count, dtype: int64

In [9]:
test_labels['toxic'].value_counts()

toxic
-1    89186
 0    57888
 1     6090
Name: count, dtype: int64

> Both test and test_labels should be merged on id column. It is mentioned in kaggle that values with -1 are not used for scoring.

In [10]:
test_labels[test_labels['toxic'] != -1].shape

(63978, 7)

In [11]:
test_labels[test_labels['toxic'] != -1].head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [12]:
test_labels[
    (test_labels['toxic'] != -1) &
    (test_labels['severe_toxic'] != -1) & 
    (test_labels['obscene'] != -1) & 
    (test_labels['threat'] != -1) & 
    (test_labels['insult'] != -1) & 
    (test_labels['identity_hate'] != -1)
].shape

(63978, 7)

**So it is clear that if one column has -1 and rest will have -1 too, so we will ignore these records for the training**